In [1]:
province_csv = "R16_LE.csv"

In [2]:
import pandas as pd
import networkx as nx
import pickle

df = pd.read_csv("Data/comuni.csv")
df2 = pd.read_csv(f"Data/{province_csv}", sep=';')
df3 = pd.read_csv("Data/italy_geo.csv")

In [3]:
df2 = df2[df2["DEST_PROV"] == df2["OR_PROV"]]
province_code = df2["DEST_PROV"].iloc[0]

In [4]:
df = df[df['provincia/codice'] == province_code]

In [5]:
df2 = df2.drop(columns = ['OR_DEST', 'OR_REG', 'OR_PROV', 'DEST_REG', 'DEST_PROV', 'TEP_TOT', 'TTP_TOT '])
df2 = df2.replace(',', '.', regex=True)
df2['KM_TOT'] = df2['KM_TOT'].astype(float)
df2 = df2[df2['KM_TOT'] > 0]
df = df[['nome', 'codice', 'popolazione']]

In [6]:
df2['OR_PROCOM'] = df2['OR_PROCOM'].map(df.set_index('codice')['nome'])
df2['DEST_PROCOM'] = df2['DEST_PROCOM'].map(df.set_index('codice')['nome'])
df = df.drop(columns = ['codice'])
df3 = df3.drop(columns = ['istat'])
df3 = df3.drop_duplicates(subset = ['comune'])

In [7]:
G = nx.from_pandas_edgelist(df2, "OR_PROCOM", "DEST_PROCOM", "KM_TOT")
population_attr = df.set_index('nome').to_dict(orient = 'index')
position_attr = df3.set_index('comune').to_dict(orient = 'index')
nx.set_node_attributes(G, population_attr)
nx.set_node_attributes(G, position_attr)

In [8]:
with open('distances.gpickle', 'wb') as f:
    pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)